In [1]:
%matplotlib notebook

In [2]:
%load_ext autoreload

%autoreload 2

In [17]:
pip install -U scikit-multiflow

Requirement already up-to-date: scikit-multiflow in /Users/venoligamage/Documents/FYP/Implementation/Asips/scikit-multiflow/src (0.6.dev0)
Note: you may need to restart the kernel to use updated packages.


# Preprocessing

In [4]:
import pandas as pd
import numpy as np

In [5]:
dataset = pd.read_csv('pulsar_data_train.csv')

In [6]:
from sklearn.impute import SimpleImputer
#imputer = SimpleImputer(missing_values = "NaN", strategy = "mean")

In [7]:
imputer = SimpleImputer(strategy="most_frequent")
print(imputer.fit_transform(dataset))

[[ 1.21156250e+02  4.83729711e+01  3.75484665e-01 ...  7.44987415e+00
   6.51592977e+01  0.00000000e+00]
 [ 7.69687500e+01  3.61755566e+01  7.12897860e-01 ...  9.41465226e+00
   1.02722975e+02  0.00000000e+00]
 [ 1.30585938e+02  5.32295335e+01  1.33408289e-01 ...  8.50836378e+00
   7.40313242e+01  0.00000000e+00]
 ...
 [ 1.16031250e+02  4.32138464e+01  6.63455691e-01 ...  1.70552145e+01
   3.12204325e+02  0.00000000e+00]
 [ 1.35664062e+02  4.99337494e+01 -8.99403060e-02 ...  7.39839490e+00
   6.23340175e+01  0.00000000e+00]
 [ 1.20726562e+02  5.04722564e+01  3.46178079e-01 ...  1.76622219e+01
   3.29548016e+02  0.00000000e+00]]


C:\Users\dell\Anaconda3\lib\site-packages\sklearn\impute\_base.py:356: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  row_mask = np.logical_not(row_mask).astype(np.bool)
C:\Users\dell\Anaconda3\lib\site-packages\sklearn\impute\_base.py:356: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  row_mask = np.logical_not(row_mask).astype(np.bool)
C:\Users\del

## Training Data

In [6]:
#Alrady tested. fast,not incrimently learn, not solve imbalance problem
from skmultiflow.data import FileStream

# 1. Create a stream

stream = FileStream('pulsar_data_train.csv')
#(./name)
print("Features:")
print(stream.feature_names)
print("Number of Targets: " + str(stream.n_targets)+"\nName: "+str(stream.target_names))
print("Target class values: " + str(stream.target_values))


Features:
['Mean of the integrated profile', 'Standard deviation of the integrated profile', 'Excess kurtosis of the integrated profile', 'Skewness of the integrated profile', 'Mean of the DM-SNR curve', 'Standard deviation of the DM-SNR curve', 'Excess kurtosis of the DM-SNR curve', 'Skewness of the DM-SNR curve']
Number of Targets: 1
Name: ['target_class']
Target class values: [0, 1]


## Testing Results

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import numpy as np
def evaluationMesures(y_true_all, y_pred_all): 
  tn, fp, fn, tp = confusion_matrix(y_true_all,y_pred_all).ravel()
  recall = recall_score(y_true_all, y_pred_all)
  precision = precision_score(y_true_all, y_pred_all)
  accuracy = accuracy_score(y_true_all, y_pred_all)
  f1 = f1_score(y_true_all, y_pred_all)
  specificity = tn/(tn + fp)
  G_mean = np.sqrt((precision * specificity))

  print('Accuracy: {0} \nRecall: {1} \nPrecision: {2} \nF1 Score: {3} \nSpecificity(TNR): {4} \nG-Mean: {5}'
      .format(accuracy,recall,precision,f1,specificity,G_mean))

## GH-EFDT

In [12]:
import datetime
from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
#fast(not much as ht),incremantly learn, not slove imbalance problem 
import numpy as np
#from skmultiflow.trees.split_criterion import GiniSplitCriterion
from skmultiflow.data import FileStream
stream = FileStream('pulsar_data_train.csv')

efdtgh = ExtremelyFastDecisionTreeClassifier(split_criterion='gaussian_hellinger')


print("info: ")
print(efdtgh.get_info)
# Setup variables to control loop and track performance
n_samples = 0
max_samples = 20000
y_true_all = []
y_pred_all = []

start_time = datetime.datetime.now()
# Train the estimator with the samples provided by the data stream
while n_samples < max_samples and stream.has_more_samples():
    X, y = stream.next_sample()
    y_pred = efdtgh.predict(X)
    y_true_all.append(y[0])
    y_pred_all.append(y_pred[0])
    efdtgh.partial_fit(X, y)
    n_samples += 1
    
    
end_time = datetime.datetime.now()
time_difference = end_time - start_time
print('{} samples analyzed.'.format(n_samples))
print('{} Time difference'.format(time_difference))
print("info: ")
print(efdtgh.get_info)

info: 
<bound method BaseSKMObject.get_info of ExtremelyFastDecisionTreeClassifier(binary_split=False, grace_period=200,
                                    leaf_prediction='nba',
                                    max_byte_size=33554432,
                                    memory_estimate_period=1000000,
                                    min_samples_reevaluate=20, nb_threshold=0,
                                    nominal_attributes=None,
                                    split_confidence=1e-07,
                                    split_criterion='gaussian_hellinger',
                                    stop_mem_management=False,
                                    tie_threshold=0.05)>
9273 samples analyzed.
0:01:51.825267 Time difference
info: 
<bound method BaseSKMObject.get_info of ExtremelyFastDecisionTreeClassifier(binary_split=False, grace_period=200,
                                    leaf_prediction='nba',
                                    max_byte_size=33554432,
    

In [13]:
evaluationMesures(y_true_all,y_pred_all)

Accuracy: 0.9674323304216542 
Recall: 0.7505882352941177 
Precision: 0.8763736263736264 
F1 Score: 0.808618504435995 
Specificity(TNR): 0.9893149709129764 
G-Mean: 0.9311334752250742


## GH-VFDT

In [14]:
import datetime
from skmultiflow.trees import HoeffdingTreeClassifier
#fast(not much as ht),incremantly learn, not slove imbalance problem 
import numpy as np
#from skmultiflow.trees.split_criterion import GiniSplitCriterion
from skmultiflow.data import FileStream
stream = FileStream('pulsar_data_train.csv')

model = HoeffdingTreeClassifier(split_criterion='gaussian_hellinger')


print("info: ")
print(model.get_info)
# Setup variables to control loop and track performance
n_samples = 0
max_samples = 20000
y_true_all = []
y_pred_all = []

start_time = datetime.datetime.now()
# Train the estimator with the samples provided by the data stream
while n_samples < max_samples and stream.has_more_samples():
    X, y = stream.next_sample()
    y_pred = model.predict(X)
    y_true_all.append(y[0])
    y_pred_all.append(y_pred[0])
    model.partial_fit(X, y)
    n_samples += 1
    
    
end_time = datetime.datetime.now()
time_difference = end_time - start_time
print('{} samples analyzed.'.format(n_samples))
print('{} Time difference'.format(time_difference))
print("info: ")
print(model.get_info)

info: 
<bound method BaseSKMObject.get_info of HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                        leaf_prediction='nba', max_byte_size=33554432,
                        memory_estimate_period=1000000, nb_threshold=0,
                        no_preprune=False, nominal_attributes=None,
                        remove_poor_atts=False, split_confidence=1e-07,
                        split_criterion='gaussian_hellinger',
                        stop_mem_management=False, tie_threshold=0.05)>
9273 samples analyzed.
0:00:01.019455 Time difference
info: 
<bound method BaseSKMObject.get_info of HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                        leaf_prediction='nba', max_byte_size=33554432,
                        memory_estimate_period=1000000, nb_threshold=0,
                        no_preprune=False, nominal_attributes=None,
                        remove_poor_atts=False, split_confidence=1e-07,
                        spli

In [15]:
evaluationMesures(y_true_all,y_pred_all)

Accuracy: 0.9583737733203925 
Recall: 0.7741176470588236 
Precision: 0.7723004694835681 
F1 Score: 0.7732079905992949 
Specificity(TNR): 0.9769678261901935 
G-Mean: 0.8686269111862857


## OnlineSMOTEBaggingClassifier

In [22]:
import datetime
from skmultiflow.meta import OnlineSMOTEBaggingClassifier
#fast(not much as ht),incremantly learn, not slove imbalance problem 
import numpy as np
#from skmultiflow.trees.split_criterion import GiniSplitCriterion
from skmultiflow.data import FileStream
stream = FileStream('pulsar_data_train.csv')

model = OnlineSMOTEBaggingClassifier()


print("info: ")
print(model.get_info)
# Setup variables to control loop and track performance
n_samples = 0
max_samples = 20000
y_true_all = []
y_pred_all = []

start_time = datetime.datetime.now()
# Train the estimator with the samples provided by the data stream
while n_samples < max_samples and stream.has_more_samples():
    X, y = stream.next_sample()
    y_pred = model.predict(X)
    y_true_all.append(y[0])
    y_pred_all.append(y_pred[0])
    model.partial_fit(X, y,np.array([0,1]))
    n_samples += 1
    
    
end_time = datetime.datetime.now()
time_difference = end_time - start_time
print('{} samples analyzed.'.format(n_samples))
print('{} Time difference'.format(time_difference))
print("info: ")
print(model.get_info)

info: 
<bound method BaseSKMObject.get_info of OnlineSMOTEBaggingClassifier(base_estimator=KNNADWINClassifier(leaf_size=30,
                                                               max_window_size=1000,
                                                               metric='euclidean',
                                                               n_neighbors=5),
                             drift_detection=True, n_estimators=10,
                             random_state=None, sampling_rate=1)>
9273 samples analyzed.
0:01:50.791627 Time difference
info: 
<bound method BaseSKMObject.get_info of OnlineSMOTEBaggingClassifier(base_estimator=KNNADWINClassifier(leaf_size=30,
                                                               max_window_size=1000,
                                                               metric='euclidean',
                                                               n_neighbors=5),
                             drift_detection=True, n_estimators=10,
  

In [23]:
evaluationMesures(y_true_all,y_pred_all)

Accuracy: 0.9681872101800928 
Recall: 0.7035294117647058 
Precision: 0.9329173166926678 
F1 Score: 0.8021462105969148 
Specificity(TNR): 0.9948949305473109 
G-Mean: 0.9634078622252028


## OnlineUnderOverBaggingClassifier

In [27]:
import datetime
from skmultiflow.meta import OnlineUnderOverBaggingClassifier
import numpy as np
from skmultiflow.data import FileStream
stream = FileStream('pulsar_data_train.csv')

model = OnlineUnderOverBaggingClassifier()


print("info: ")
print(model.get_info)
# Setup variables to control loop and track performance
n_samples = 0
max_samples = 9000
y_true_all = []
y_pred_all = []
wait_samples = 300

start_time = datetime.datetime.now()
# Train the estimator with the samples provided by the data stream
while n_samples < max_samples and stream.has_more_samples():
    X, y = stream.next_sample()
    y_pred = model.predict(X)
    if (n_samples > wait_samples):
        y_true_all.append(y[0])
        y_pred_all.append(y_pred[0])
    model.partial_fit(X, y,np.array([0,1]))
    n_samples += 1
    
    
end_time = datetime.datetime.now()
time_difference = end_time - start_time
print('{} samples analyzed.'.format(n_samples))
print('{} Time difference'.format(time_difference))
print("info: ")
print(model.get_info)

info: 
<bound method BaseSKMObject.get_info of OnlineUnderOverBaggingClassifier(base_estimator=KNNADWINClassifier(leaf_size=30,
                                                                   max_window_size=1000,
                                                                   metric='euclidean',
                                                                   n_neighbors=5),
                                 drift_detection=True, n_estimators=10,
                                 random_state=None, sampling_rate=2)>
9000 samples analyzed.
0:01:20.830538 Time difference
info: 
<bound method BaseSKMObject.get_info of OnlineUnderOverBaggingClassifier(base_estimator=KNNADWINClassifier(leaf_size=30,
                                                                   max_window_size=1000,
                                                                   metric='euclidean',
                                                                   n_neighbors=5),
                              

In [25]:
evaluationMesures(y_true_all,y_pred_all)

Accuracy: 0.9692656098350049 
Recall: 0.7941176470588235 
Precision: 0.8598726114649682 
F1 Score: 0.8256880733944956 
Specificity(TNR): 0.9869405200047489 
G-Mean: 0.9212182815690736


In [28]:
evaluationMesures(y_true_all,y_pred_all)

Accuracy: 0.9701115070697781 
Recall: 0.7934508816120907 
Precision: 0.8677685950413223 
F1 Score: 0.8289473684210527 
Specificity(TNR): 0.9878557874762809 
G-Mean: 0.9258672846589469
